## Loading the model

In [1]:
from src import PathOrganizer
from src import ExploratoryFunctions as es

import pandas as pd
import cobra
from cobra.flux_analysis import flux_variability_analysis as fva

pd.set_option("display.max_rows", 500)

Hello World


## Media Dictionaries

In [2]:
kwoji = {
    "EX_cpd00001_e0": (-1000, 1000),
    "EX_cpd00009_e0": (-60, 1000),
    "EX_cpd00011_e0": (-2, 1000),
    "EX_cpd00013_e0": (-40, 1000),
    "EX_cpd00023_e0": (-2, 1000),
    "EX_cpd00028_e0": (-0.05, 1000),
    "EX_cpd00029_e0": (-50, 1000),
    "EX_cpd00030_e0": (-0.25, 1000),
    "EX_cpd00033_e0": (-1, 1000),
    "EX_cpd00035_e0": (-1, 1000),
    "EX_cpd00039_e0": (-1, 1000),
    "EX_cpd00041_e0": (-2, 1000),
    "EX_cpd00048_e0": (-76.35, 1000),
    "EX_cpd00051_e0": (-1, 1000),
    "EX_cpd00053_e0": (-2, 1000),
    "EX_cpd00054_e0": (-1, 1000),
    "EX_cpd00060_e0": (-1, 1000),
    "EX_cpd00065_e0": (-1, 1000),
    "EX_cpd00066_e0": (-1, 1000),
    "EX_cpd00067_e0": (-1000, 1000),
    "EX_cpd00069_e0": (-1, 1000),
    "EX_cpd00084_e0": (-2, 1000),
    "EX_cpd00092_e0": (-0.1, 1000),
    "EX_cpd00099_e0": (-20, 1000),
    "EX_cpd00104_e0": (-0.005, 1000),
    "EX_cpd00107_e0": (-1, 1000),
    "EX_cpd00119_e0": (-2.5, 1000),
    "EX_cpd00129_e0": (-1, 1000),
    "EX_cpd00132_e0": (-2, 1000),
    "EX_cpd00156_e0": (-1, 1000),
    "EX_cpd00161_e0": (-1, 1000),
    "EX_cpd00182_e0": (-0.1, 1000),
    "EX_cpd00184_e0": (-0.1, 1000),
    "EX_cpd00205_e0": (-90, 1000),
    "EX_cpd00218_e0": (-0.001, 1000),
    "EX_cpd00220_e0": (-0.001, 1000),
    "EX_cpd00254_e0": (-25, 1000),
    "EX_cpd00305_e0": (-0.001, 1000),
    "EX_cpd00311_e0": (-0.1, 1000),
    "EX_cpd00322_e0": (-2, 1000),
    "EX_cpd00393_e0": (-0.001, 1000),
    "EX_cpd00588_e0": (-10, 1000),
    "EX_cpd00644_e0": (-0.001, 1000),
    "EX_cpd00971_e0": (-100, 1000),
    "EX_cpd01217_e0": (-0.1, 1000),
    "EX_cpd10515_e0": (-0.2, 1000),
    "EX_cpd11606_e0": (-0.01, 1000),
}

closed_uptake = [
    "EX_cpd00007_e0",  # Oxygen
]

essential = [
    "EX_cpd00034_e0",  # Zn2+
    "EX_cpd00058_e0",  # Cu2+
    "EX_cpd00063_e0",  # Ca2+
    "EX_cpd00149_e0",  # Co2+
    "EX_cpd10516_e0",  # fe3
]

c_sources = [
    "EX_cpd00027_e0",  # glucose
    "EX_cpd01200_e0",  # palatinose
    "EX_cpd00076_e0",  # sucrose
    "EX_cpd00082_e0",  # fructose
]

## All Relevant Variables

In [3]:
lmur_pathorg = PathOrganizer()
model_dir = lmur_pathorg.get_model_version_path(mo_name="LigilactobacillusMurinus")
model = lmur_pathorg.load_model(model_dir)

Available Model Versions:
0) 0.0.0.sbml
1) 1.1.1.sbml
2) 1.1.2.sbml
3) 1.1.3.sbml
4) 1.1.4.sbml
5) 1.1.5.sbml
6) 2.1.1.sbml
7) 2.2.1.sbml
8) 2.2.2.sbml
9) 2.2.3.sbml
10) 2.2.4.sbml
11) 2.2.5.sbml
12) 2.2.6.sbml
13) 2.2.7.sbml
14) pickles


In [4]:
exploration = es()
exploration.set_media(
    model=model, medium=kwoji, essential=essential, closed=closed_uptake
)
u, s = exploration.gather_media_fluxes(model=model)

dif_u = exploration.find_differences(target_df=u, medium=kwoji)
dif_s = exploration.find_differences(target_df=s, medium=kwoji)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


## Results

In [5]:
exploration.test_carbon_sources(model=model, carbon_list=c_sources)

,Met. Name,FBA Result,Flux
w/o C-Source,NaN,0.096800,NaN
EX_cpd00027_e0,D-Glucose-e0,0.096796,-0.22466
EX_cpd01200_e0,Palatinose-e0,0.096796,-0.11233
EX_cpd00076_e0,Sucrose-e0,0.096796,-0.11233
EX_cpd00082_e0,D-Fructose-e0,0.096796,-0.22432


In [19]:
def test():
    kwoji_list = list(kwoji.keys())
    for c in c_sources:
        exploration.set_media(
        model=model, medium=kwoji, essential=essential, closed=closed_uptake
    )
        with model:
            c_tgt = model.reactions.get_by_id(c)
            c_tgt.bounds = (-1000,1000)
            print(f"{c_tgt.id} --> {c_tgt.name}")
            result_fva = fva(model=model, reaction_list=kwoji_list,fraction_of_optimum=0.95)
            display(result_fva)

In [20]:
test()

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


EX_cpd00027_e0 --> D-Glucose-e0 Exchange


,minimum,maximum
EX_cpd00001_e0,-9.132214e+02,1.717277e+02
EX_cpd00009_e0,-6.000000e+01,-1.233710e-01
EX_cpd00011_e0,-1.896663e+00,1.000000e+03
EX_cpd00013_e0,-4.000000e+01,1.797401e+01
EX_cpd00023_e0,-2.000000e+00,1.939422e+00
EX_cpd00028_e0,-5.000000e-04,1.795766e-01
EX_cpd00029_e0,-5.000000e+01,6.986146e+02
EX_cpd00030_e0,-7.060625e-04,-6.707594e-04
EX_cpd00033_e0,-1.000000e+00,4.404130e-01
EX_cpd00035_e0,-1.000000e+00,5.697401e+01


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


EX_cpd01200_e0 --> Palatinose-e0 exchange


...

EX_cpd00076_e0 --> Sucrose-e0 Exchange


,minimum,maximum
EX_cpd00001_e0,-1.000000e+03,1.239050e+02
EX_cpd00009_e0,-6.000000e+01,-1.233710e-01
EX_cpd00011_e0,-1.896663e+00,1.000000e+03
EX_cpd00013_e0,-4.000000e+01,1.797401e+01
EX_cpd00023_e0,-2.000000e+00,1.939422e+00
EX_cpd00028_e0,-5.000000e-04,1.795766e-01
EX_cpd00029_e0,-5.000000e+01,6.791130e+02
EX_cpd00030_e0,-7.060625e-04,-6.707594e-04
EX_cpd00033_e0,-1.000000e+00,4.404130e-01
EX_cpd00035_e0,-1.000000e+00,5.697401e+01


Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


EX_cpd00082_e0 --> D-Fructose-e0 Exchange


,minimum,maximum
EX_cpd00001_e0,-9.132214e+02,1.717277e+02
EX_cpd00009_e0,-6.000000e+01,-1.233710e-01
EX_cpd00011_e0,-1.896663e+00,1.000000e+03
EX_cpd00013_e0,-4.000000e+01,1.797401e+01
EX_cpd00023_e0,-2.000000e+00,1.939422e+00
EX_cpd00028_e0,-5.000000e-04,1.795766e-01
EX_cpd00029_e0,-5.000000e+01,7.338081e+02
EX_cpd00030_e0,-7.060625e-04,-6.707594e-04
EX_cpd00033_e0,-1.000000e+00,4.404130e-01
EX_cpd00035_e0,-1.000000e+00,5.697401e+01


## Code Guidelines and Whatnot